<a href="https://colab.research.google.com/github/ayadassouki/ayadassouki-ACL-Task-A-Sentiment-Analysis/blob/main/distillbert(Pre_fine_tuning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the DistilBERT tokenizer and model for sequence classification
model_name = "distilbert-base-uncased"  # DistilBERT model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

# Sample text for testing
text = "I just got the best news today! I'm so excited and happy."

# Prepare the input for DistilBERT
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

# Make the prediction
with torch.no_grad():
    outputs = model(**inputs)

# Get logits (raw predictions)
logits = outputs.logits

# Apply a sigmoid to convert logits to probabilities
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits)

# Define emotion labels corresponding to the 5 outputs
emotion_labels = ["Anger", "Fear", "Joy", "Sadness", "Surprise"]

# Convert probabilities to binary values (1 if > 0.5, else 0)
predictions = (probs > 0.5).int()

# Print the results as a dictionary
result = {emotion_labels[i]: predictions[0][i].item() for i in range(len(emotion_labels))}
print(result)


In [ ]:
import pandas as pd
data = pd.read_csv('eng.csv')
data.drop(['id'], axis=1, inplace=True)
data.head()

In [ ]:
text_data = data['text'].tolist()
len(text_data)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the DistilBERT tokenizer and model for sequence classification
model_name = "distilbert-base-uncased"  # DistilBERT model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)
prediction_list = []
i = 0
for text in text_data:
  # Prepare the input for DistilBERT
  inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

  # Make the prediction
  with torch.no_grad():
      outputs = model(**inputs)

  # Get logits (raw predictions)
  logits = outputs.logits

  # Apply a sigmoid to convert logits to probabilities
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(logits)

  # Define emotion labels corresponding to the 5 outputs
  emotion_labels = ["Anger", "Fear", "Joy", "Sadness", "Surprise"]

  # Convert probabilities to binary values (1 if > 0.5, else 0)
  predictions = (probs > 0.5).int()

  # Print the results as a dictionary
  result = {emotion_labels[i]: predictions[0][i].item() for i in range(len(emotion_labels))}
  prediction_list.append(result)
  print(f"predictions made: {i}/2768")
  i = i+1

len(predictions)


In [ ]:
len(prediction_list)

In [ ]:
result_df = pd.DataFrame(prediction_list)
result_df.head()

In [ ]:
result_df.rename(columns={'Anger': 'p_anger', 'Fear': 'p_fear', 'Joy': 'p_joy', 'Sadness': 'p_sadness', 'Surprise': 'p_surprise'}, inplace=True)
result_df.head()

In [ ]:
result_df['text'] = data['text'].values
result_df.head()

In [ ]:
# Columns to add
columns_to_add = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']

# Iterate over columns_to_add and set the corresponding values from val_df
for col in columns_to_add:
    result_df[col] = data[col].values

# Display the updated v_df
result_df.head()

In [ ]:
from sklearn.metrics import classification_report

# Assuming you have true labels in the DataFrame as 'true_anger', 'true_fear', etc.
true_labels = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
pred_emotion = ['p_anger', 'p_fear', 'p_joy', 'p_sadness', 'p_surprise']

# Create an empty dictionary to store classification reports for each emotion
classification_reports = {}

# Iterate through each emotion and calculate the classification report
for true_label, pred_label in zip(true_labels, pred_emotion):
    true_values = result_df[true_label]  # True labels for the emotion
    pred_values = result_df[pred_label]  # Predicted labels for the emotion


    # Generate the classification report
    report = classification_report(true_values, pred_values)

    # Store the report in the dictionary
    classification_reports[true_label] = report

# Print the classification reports
for emotion, report in classification_reports.items():
    print(f"Classification Report for {emotion}:")
    print(report)
    print("\n")